In [1]:
%%file myConv.py

import torch
import numpy as np
def myConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #groups exeption соединения между входами и выходами проверка корректности
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    # заполнение padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad2d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad2d(2)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad2d(padding)
      matrix = pad(matrix)

    #генерация ядра случайными значениями
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size)

    #filter = b
    # bias_val = bi

    llist = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
        for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
          summa = 0
          for c in range (in_channels//groups): #groups
            if groups>1:
              val = matrix[l*(in_channels//groups)+c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            else:
              val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*dilation+1:dilation]
            mini_sum = (val*filter[l][c]).sum()
            summa = summa + mini_sum
          feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      llist.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))

    return np.array(llist), np.array(filter), np.array(bias_val)

  return Svertca

Writing myConv.py


In [2]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Writing conftest.py


In [3]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/SMZ/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor1 = torch.rand(4, 5, 6)
tensor2 = torch.rand(3, 28, 28)
tensor3 = torch.rand(1, 1, 1)


def test_1(session):
  myFunction = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  torchFunction.weight.data = torch.tensor(kernel)
  torchFunction.bias.data = torch.tensor(bias_val)
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  torchFunction2.weight.data = torch.tensor(kernel2)
  torchFunction2.bias.data = torch.tensor(bias_val2)
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  torchFunction3.weight.data = torch.tensor(kernel3)
  torchFunction3.bias.data = torch.tensor(bias_val3)
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult



Writing test_http.py


In [4]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.11.6, pytest-7.4.3, pluggy-1.3.0
rootdir: d:\SMZ\1
collected 3 items

test_http.py ...                                                         [100%]

============================== 3 passed in 6.26s ==============================


In [5]:
import numpy as np
import torch
import sys
import os

py_file_location = "/SMZ/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv2d

tensor4 = torch.rand(4, 5, 5)

myFunction4 = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
result4, kernel4, bias_val4 = myFunction4(tensor4)
torchFunction4 = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction4.weight.data = torch.tensor(kernel4)
torchFunction4.bias.data = torch.tensor(bias_val4)

print(result4)
print('===================================')
print(np.array(torchFunction4(tensor4).data))

[[[0.86246848 1.98532915 1.86449194 1.37985706]
  [1.98610306 5.03702164 4.67544556 2.23135519]
  [1.75034809 5.42129898 5.85383797 2.90122819]
  [1.48679781 2.73089051 3.04978895 1.80450749]]

 [[1.37376678 2.20550013 2.30822754 1.31496203]
  [2.82773757 5.97429848 5.30568266 1.95677114]
  [2.86085105 4.39055967 5.27298594 2.16799593]
  [1.33667779 1.96517229 1.74234831 1.03072596]]]
[[[0.8624685 1.9853292 1.8644918 1.3798571]
  [1.9861029 5.0370226 4.6754456 2.2313552]
  [1.7503481 5.421299  5.853838  2.9012282]
  [1.4867978 2.7308908 3.049789  1.8045075]]

 [[1.3737668 2.2055004 2.3082275 1.314962 ]
  [2.8277376 5.9742985 5.305682  1.9567711]
  [2.860851  4.390559  5.2729855 2.1679962]
  [1.3366778 1.9651723 1.7423483 1.030726 ]]]
